In [ ]:
### Proyecto ###

###El programa registra los movimientos contables de una empresa y pasa los saldos  de las diferentes cuentas a una base de datos.

####

import mysql.connector

try:
    mibase=mysql.connector.connect(
    host="localhost",
    user="root",
    password="")

    micursor=mibase.cursor()
    micursor.execute("CREATE DATABASE cuentas")
except:
    pass


In [ ]:
###La primera vez que se utiliza se crea la base de datos y tabla automaticamente, cuando se corre las veces subsiguientes no lanzara error de base ya existente por el try/except.

try:
    mibase=mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="cuentas")

    micursor=mibase.cursor()

    micursor.execute("CREATE TABLE cuentas (id INT AUTO_INCREMENT PRIMARY KEY, nombre VARCHAR(50),saldo INT)")


    sql="INSERT INTO cuentas(nombre,saldo) VALUES(%s,%s)"
    datos=[("efectivo",0),("mercaderias",0),("proveedores",0),("ventas",0),("CMV",0),("deudores",0),("capital",0)]
    micursor.executemany(sql,datos)
    mibase.commit()
except: 
    pass

In [ ]:


mibase=mysql.connector.connect(
host="localhost",
user="root",
password="",
database="cuentas")

micursor=mibase.cursor()


sql_saldo_efectivo="SELECT saldo FROM cuentas WHERE nombre='efectivo'"
micursor.execute(sql_saldo_efectivo)
resultado=micursor.fetchone()
saldo_efectivo=resultado[0]
sql_saldo_mercaderias="SELECT saldo FROM cuentas WHERE nombre='mercaderias'"
micursor.execute(sql_saldo_mercaderias)
resultado=micursor.fetchone()
saldo_mercaderias=resultado[0]
sql_saldo_proveedores="SELECT saldo FROM cuentas WHERE nombre='proveedores'"
micursor.execute(sql_saldo_proveedores)
resultado=micursor.fetchone()
saldo_proveedores=resultado[0]
sql_saldo_ventas="SELECT saldo FROM cuentas WHERE nombre='ventas'"
micursor.execute(sql_saldo_ventas)
resultado=micursor.fetchone()
saldo_ventas=resultado[0]
sql_saldo_cmv="SELECT saldo FROM cuentas WHERE nombre='CMV'"
micursor.execute(sql_saldo_cmv)
resultado=micursor.fetchone()
saldo_cmv=resultado[0]
sql_saldo_deudores="SELECT saldo FROM cuentas WHERE nombre='deudores'"
micursor.execute(sql_saldo_deudores)
resultado=micursor.fetchone()
saldo_deudores=resultado[0]
sql_saldo_capital="SELECT saldo FROM cuentas WHERE nombre='capital'"
micursor.execute(sql_saldo_capital)
resultado=micursor.fetchone()
saldo_capital=resultado[0]

saldo_resultados=saldo_ventas-saldo_cmv

###Cada cuenta es un objeto con los atributos nombre y saldo. Cada objeto tiene una funcion de adicion y sustraccion.

class cuenta():
    def __init__(self, nombre, saldo):
        self.nombre=nombre
        self.saldo=float(saldo)
        
    def adicion(self,x):
        self.saldo=self.saldo+float(x)
    
    def sustraccion(self,x):
        self.saldo=self.saldo-float(x)
        
        

efectivo=cuenta('Efectivo',float(saldo_efectivo))

mercaderias=cuenta('Mercaderias',float(saldo_mercaderias))

proveedores=cuenta('Proveedores',float(saldo_proveedores))

ventas=cuenta('Ventas',float(saldo_ventas))

costo_mercaderias=cuenta('CMV',float(saldo_cmv))

deudores=cuenta('Deudores',float(saldo_deudores))

capital=cuenta('Capital',float(saldo_capital))

resultados=cuenta('Resultados',(saldo_resultados))

###Ante cada operacion el sistema modifica al menos dos cuentas diferentes siguiendo el principio de partida doble. Por ejemplo, en una compra en efectivo el monto se resta del saldo de la cuenta "efectivo" y se suma el mismo monto a la cuenta "mercaderias".

###Todas estas modificaciones se hacen tambien en la base de datos. 


def balance():
    print('''
    
    ESTADO DE SITUACION PATRIMONIAL
    
    ACTIVO
    
    Efectivo:            {}
    Deudores por ventas: {}
    Mercaderias:         {}
    ---------------------------
    Total activo:        {}
    
    PASIVO Y PN
    
    Proveedores:         {}
    ---------------------------
    
    Total pasivo:        {}
    
    Capital:             {}
    Resultados           {}
    ---------------------------
    ---------------------------
    Total Pasivo y PN:   {}
    
    '''.format(efectivo.saldo,deudores.saldo,mercaderias.saldo,(efectivo.saldo+deudores.saldo+mercaderias.saldo),proveedores.saldo,proveedores.saldo,capital.saldo,resultados.saldo,(proveedores.saldo+capital.saldo+resultados.saldo)))

programa_on=True

###Para simplificar, se limitaron a 7 las acciones basicas del universo amplio existente de registraciones contables.

###Se hacen varios supuestos a fines de simplificar el funcionamiento:
###- La empresa no invierte los excedentes de efectivo.
###- Las compras y ventas a credito no generan interes
###-El margen de ganancia de los productos es del 10%

###Mediante try/except se manejan los errores en caso de que el usuario escriba una opcion no valida, es decir que no figure en el menu, o un valor no numerico en las operaciones.


def programa():
    while programa_on:
        print('''
    
    SISTEMA DE GESTION DE CONTABILIDAD
    
    1. Compras a contado
    2. Compras a credito
    3. Ventas a contado
    4. Ventas a credito
    5. Pago a proveedores
    6. Pago de clientes deudores
    7. Aporte de capital
    ----------------------------
    8. Balance
    9. Estado de resultados
    
    ---------------------------
    
    B: Borrar todos los datos
    
    0. Salir
    
    
        ''')
        opcion=input("Ingrese la opcion a registrar")
        if opcion=='1':
            print(' Compra a contado')
            monto=input('Ingrese monto de la compra')
            try:
                if float(efectivo.saldo)>=float(monto):
                    efectivo.sustraccion(monto)
                    mercaderias.adicion(monto)
                    sql_efectivo="UPDATE cuentas SET saldo= {} WHERE nombre='efectivo'".format(efectivo.saldo)
                    micursor.execute(sql_efectivo)
                    mibase.commit()
                    sql_mercaderias="UPDATE cuentas SET saldo= {} WHERE nombre='mercaderias'".format(mercaderias.saldo)
                    micursor.execute(sql_mercaderias)
                    mibase.commit()
                    print('Compra realizada en efectivo por ${}'.format(monto))
                else:
                    print('Efectivo insuficiente')
            except:
                print('El monto debe ser numerico')
                pass
                
        elif opcion=='2':
            print('Compra a credito')
            monto=input('Ingrese monto de la compra')
            try:
                proveedores.adicion(monto)
                mercaderias.adicion(monto)
                sql_proveedores="UPDATE cuentas SET saldo= {} WHERE nombre='proveedores'".format(proveedores.saldo)
                micursor.execute(sql_proveedores)
                mibase.commit()
                sql_mercaderias="UPDATE cuentas SET saldo= {} WHERE nombre='mercaderias'".format(mercaderias.saldo)
                micursor.execute(sql_mercaderias)
                mibase.commit()
            except:
                print('El monto debe ser numerico')
                pass
        elif opcion=='3':
            try:
                print('Venta a contado')
                monto=float(input('Ingrese monto de la venta'))
                ventas.adicion(monto)
                efectivo.adicion(monto)
                margen=monto*0.9
                mercaderias.sustraccion(float(margen))
                costo_mercaderias.adicion(float(margen)) 
                resultados.adicion(monto-margen)
                sql_ventas="UPDATE cuentas SET saldo= {} WHERE nombre='ventas'".format(ventas.saldo)
                micursor.execute(sql_ventas)
                mibase.commit()
                sql_efectivo="UPDATE cuentas SET saldo= {} WHERE nombre='efectivo'".format(efectivo.saldo)
                micursor.execute(sql_efectivo)
                mibase.commit()
                sql_mercaderias="UPDATE cuentas SET saldo= {} WHERE nombre='mercaderias'".format(mercaderias.saldo)
                micursor.execute(sql_mercaderias)
                mibase.commit()
                sql_cmv="UPDATE cuentas SET saldo= {} WHERE nombre='cmv'".format(costo_mercaderias.saldo)
                micursor.execute(sql_cmv)
                mibase.commit()
            except:
                print('El monto debe ser numerico')
                pass
        elif opcion=='4':
            print('Venta a credito')
            monto=float(input('Ingrese monto de la venta'))
            try:
                ventas.adicion(monto)
                deudores.adicion(monto)
                margen=monto*0.9
                mercaderias.sustraccion(float(margen))
                costo_mercaderias.adicion(float(margen)) 
                resultados.adicion(monto-margen)
                sql_deudores="UPDATE cuentas SET saldo= {} WHERE nombre='deudores'".format(deudores.saldo)
                micursor.execute(sql_deudores)
                mibase.commit()
                sql_ventas="UPDATE cuentas SET saldo= {} WHERE nombre='ventas'".format(ventas.saldo)
                micursor.execute(sql_ventas)
                mibase.commit()
                sql_mercaderias="UPDATE cuentas SET saldo= {} WHERE nombre='mercaderias'".format(mercaderias.saldo)
                micursor.execute(sql_mercaderias)
                mibase.commit()
                sql_cmv="UPDATE cuentas SET saldo= {} WHERE nombre='cmv'".format(costo_mercaderias.saldo)
                micursor.execute(sql_cmv)
                mibase.commit()
            except:
                print('El monto debe ser numerico')
                pass
        elif opcion=='5':
            print('Pago a proveedores')
            try:
                monto=float(input('Ingrese monto a pagar'))
                if monto <=float(efectivo.saldo):
                    efectivo.sustraccion(monto)
                    proveedores.sustraccion(monto)
                    sql_efectivo="UPDATE cuentas SET saldo= {} WHERE nombre='efectivo'".format(efectivo.saldo)
                    micursor.execute(sql_efectivo)
                    mibase.commit()
                    sql_proveedores="UPDATE cuentas SET saldo= {} WHERE nombre='proveedores'".format(proveedores.saldo)
                    micursor.execute(sql_proveedores)
                    mibase.commit()
                else:
                    print('No hay efectivo suficiente para pagar esta cuenta')
            except:
                print('El monto debe ser numerico')
                pass
            
               
        elif opcion=='6':
            print('Pago de clientes deudores')
            monto=input('Ingrese monto recibido')
            try:
                efectivo.adicion(monto)
                deudores.sustraccion(monto)
                sql_efectivo="UPDATE cuentas SET saldo= {} WHERE nombre='efectivo'".format(efectivo.saldo)
                micursor.execute(sql_efectivo)
                mibase.commit()
                sql_deudores="UPDATE cuentas SET saldo= {} WHERE nombre='deudores'".format(deudores.saldo)
                micursor.execute(sql_deudores)
                mibase.commit()
            except:
                print('El monto debe ser numerico')
                pass
        elif opcion=='7':
            print('Aporte de capital en efectivo')
            monto=input('Ingrese monto de capital aportado')
            try:
                capital.adicion(monto)
                efectivo.adicion(monto)
                sql_capital="UPDATE cuentas SET saldo= {} WHERE nombre='capital'".format(capital.saldo)
                micursor.execute(sql_capital)
                mibase.commit()
                sql_efectivo="UPDATE cuentas SET saldo= {} WHERE nombre='efectivo'".format(efectivo.saldo)
                micursor.execute(sql_efectivo)
                mibase.commit()
            except:
                print('El monto debe ser numerico')
                pass
        elif opcion=='8':
            balance()
        elif opcion=='9':
            print('Estado de resultados')
            print('''
            Ventas             {}
            Costos de ventas   {}
            ------------------------
            Resultado          {}'''.format(ventas.saldo,costo_mercaderias.saldo,(ventas.saldo-costo_mercaderias.saldo)))
        elif opcion=='b' or opcion=='B':
            a=input('Se borraran todos los datos. ¿ Desea continuar ? S/N')
            if a=='s' or a=='S':
                sql_efectivo="UPDATE cuentas SET saldo= 0 WHERE nombre='efectivo'"
                micursor.execute(sql_efectivo)
                mibase.commit()
                sql_capital="UPDATE cuentas SET saldo= 0 WHERE nombre='capital'"
                micursor.execute(sql_capital)
                mibase.commit()
                sql_mercaderias="UPDATE cuentas SET saldo=0 WHERE nombre='mercaderias'"
                micursor.execute(sql_mercaderias)
                mibase.commit()
                sql_deudores="UPDATE cuentas SET saldo= 0 WHERE nombre='deudores'"
                micursor.execute(sql_deudores)
                mibase.commit()
                sql_proveedores="UPDATE cuentas SET saldo= 0 WHERE nombre='proveedores'"
                micursor.execute(sql_proveedores)
                mibase.commit()
                sql_ventas="UPDATE cuentas SET saldo= 0 WHERE nombre='ventas'"
                micursor.execute(sql_ventas)
                mibase.commit()
                sql_cmv="UPDATE cuentas SET saldo= 0 WHERE nombre='cmv'"
                micursor.execute(sql_cmv)
                mibase.commit()
                resultados.saldo=0
                print('Se han borrado los datos de la base. Por favor reinicie el programa para actualizar los datos')
                break
            elif a=='N' or a=='n':
                    print('No se han modificado los datos')
                    pass
            else:
                    print('Respuesta no valida')
                    pass


        elif opcion=='0':
            print('Gracias por usar el sistema de gestion')
            
            break
        else:
            pass


###Mediante la opcion 8 y 9 se puede mostrar el balance y el estado de resultados respectivamente. 

###Mediante la opcion 'b' se borran todos los datos de la base de datos. El sistema requiere una confirmacion del usuario para hacer esto.

###El programa sigue pidiendo inputs del usuario mediante el while, hasta que es terminado con la opcion '0'.


In [ ]:
programa()